In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm as normal, binom
import bplot as bp
from scipy.optimize import minimize
from scipy.special import loggamma
import patsy

In [2]:
def ll_normal(beta, yX):
    y = yX[:, 0]
    X = yX[:, 1:]
    N = X.shape[0]
    mu = np.full(N, np.nan)
    for n in range(N):
        mu[n] = np.sum(X[n, :] * beta)
        
    d = y - mu
    return np.sum(d*d)

def optim(data, initval = None):
    k = data.shape[1]-1
    return minimize(ll_normal, (initval if initval else np.random.normal(size = k)), args=(data), method="BFGS")["x"]

def bootstrap(data, R, fun):
    N, k = data.shape
    k -= 1
    thetas = np.full((R, k), np.nan)
    for r in range(R):
        idx = np.random.choice(N, N, replace=True)
        thetas[r, :] = fun(data[idx, :])
    return thetas

# 01 November, 2019

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/roualdes/data/master/possum.csv")
X = patsy.dmatrix(" ~C(pop)", data=df)
yX = np.c_[df["totalL"], X]

## The Model
$$Y_n \sim N(\mu_n,\sigma^2)$$
$$\mu_n=\beta_0 + \beta_1*x_n$$

In [5]:
mus = bootstrap(yX, 1001, optim)

In [12]:
np.percentile(mus, [2.5, 97.5], axis=0).T

array([[85.82295085, 87.76206903],
       [-1.02385213,  2.22041797]])

In [18]:
np.percentile(mus.sum(axis=1), [2.5, 97.5])

array([86.06363645, 88.77500018])